## * Recommendation project for people who want to open a restaurant in Berlin,Germany.

#### 1) Introduction/Business Problem 

* **Introduction**
= Berlin is the one of the very trendy cities. Berlin has been rapidly changing a lot because many people are coming for leaving and working. Hence various resaurants are openining and closing. my purpose of the data analysis is to help people's business. 


* **Problem**
= Many people can possibly think that once they open a restaurant with very fancy and tasty food in Berlin, wherever it opens, it would be a success. but the problem is that in Berlin, there are already too many competitors on even one kind of food. Personally, I already saw some people struggling to find a proper solution or information to start with their business. but people tend to rely on the some information in internet, which is not reliable. 


* **Objective**
= Hence I would like to help people's business to avoid the worst scenario. Because once people starts with a own business, probably they might need to spend the cost to setup the restaurant.Therefore , I'm going to design the recommendation data analysis by using the Foursqure data and clustering technique,ML,  And Based on the data, I'm going to recommend the best spot for their business. Kreuzberg is currently quite trendy and popular town in Berlin for people to enjoy various cultures like foods. Therefore I'm going to focus on this town to start with this project. 


* **target audience**
= Even though this project is desigined for only people who want to open a restaurant in Kreuzberg of Berlin, Actually this approach way can be applied to anywhere.  

#### 2) Data

There are two big categories for the data, which are location data and venue data from Foursquare. these data will be utilized for the data analaysis when to recommend the places for users who who want to find out the nice places for their businesses.  

1. Berlin post codes, geo-location information (latitude, longitude)
: basically, the data includes geo location information of all berlin areas as well as post code information.
https://data.technologiestiftung-berlin.de/dataset/plz/en


2. Berlin traffic information 
: this data-set provides traffic information per street.
https://data.technologiestiftung-berlin.de/dataset/verkehrsmengen/en


2. Berlin map based on postcode information (to narrow down the range )
: actually the data is not utilized on the real code, however it is referred to narrow down the data range(area)
https://www.interkart.de/en/wall-maps/europe/germany/city-maps/berlin-postcode-area-map.html  


3. Berlin Kreuzberg map information to check
http://zip-code.en.mapawi.com/germany/10/kreisfreie-stadt-berlin/2/141/berlin-kreuzberg/10961/1330/


4. Foursquare venue information ( via API )
https://developer.foursquare.com/docs/api/venues/details

* test ( checking out the data )
= this data comes from the first data set, which is berlin post code information. In order to narrow down the data-set scale,I'll filter out other town information. 

In [10]:
import json
import pandas as pd
lst_dict = []


with open('./data/postcodes_berlin.geojson') as f:
    data = json.load(f)

df = pd.DataFrame(columns=['postcode','latitude','longitude'])

for feature in data['features']:
    # only check Kreuzberg area in Berlin 
    if feature['properties']['spatial_name'] in ('10961', '10963', '10965', '10967', '10969', '10997', '10999'):  
        for position in feature['geometry']['coordinates']:
            for i in position:
                df = df.append({'postcode': feature['properties']['spatial_name'],'latitude':i[1],'longitude':i[0]}, ignore_index=True)

df.shape

(1361, 3)

* Extracting only Kreuzberg of Berlin's post code information and getting unique values by groupping the post code data

In [38]:
df_group=df.groupby('postcode').mean().reset_index()
df_group

,postcode,latitude,longitude
0,10961,52.491796,13.394398
1,10963,52.499171,13.384151
2,10965,52.487955,13.389993
3,10967,52.491046,13.419179
4,10969,52.502517,13.401676
5,10997,52.500781,13.435210
6,10999,52.499876,13.421885


* Foursquare venue information based on the area,"Berlin, Kreuzberg"

In [45]:
CLIENT_ID = 'BDLXZJQYJRJT03ITMAD0QYWJC1SDRS124RRYJ1M1O2JEFOR4' # your Foursquare ID
CLIENT_SECRET = 'VLAQW1AH2IJE4WVSP34XUGNY00OMJPWDBNGSCNF4FQCKNO3T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
import requests

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            30)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
berlin_venues = getNearbyVenues(names=df_group['postcode'],
                                   latitudes=df_group['latitude'],
                                   longitudes=df_group['longitude']
                                  )



10961
10963
10965
10967
10969
10997
10999


In [48]:
berlin_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,10961,52.491796,13.394398,Nonne & Zwerg,52.492641,13.396822,Mediterranean Restaurant
1,10961,52.491796,13.394398,Ocakbasi Holzkohlengrill Restaurant,52.493043,13.394863,Mediterranean Restaurant
2,10961,52.491796,13.394398,Space Hall,52.490746,13.394768,Record Shop
3,10961,52.491796,13.394398,Cuccuma,52.490682,13.394192,Coffee Shop
4,10961,52.491796,13.394398,Chapter One,52.489762,13.395976,Coffee Shop
...,...,...,...,...,...,...,...
199,10999,52.499876,13.421885,Biererei Bar & Vintage Cellar,52.500666,13.421093,Beer Bar
200,10999,52.499876,13.421885,Fahimi,52.499149,13.419708,Cocktail Bar
201,10999,52.499876,13.421885,Smyrna Kuruyemis,52.501141,13.419901,Restaurant
202,10999,52.499876,13.421885,Lasan,52.499842,13.418158,Kurdish Restaurant


* **average daily traffic data**

1) str_name = street name

2) dtv = average daily traffic count 

In [67]:
import json
import pandas as pd
lst_dict = []


with open('./data/traffic_count_2014.geojson') as f:
    data_traffic = json.load(f)

df_traffic = pd.DataFrame(columns=['str_name','dtv','latitude','longitude'])
    
for feature in data_traffic['features']:
    # only check Kreuzberg area in Berlin 
    #if feature['properties']['spatial_name'] in ('10961', '10963', '10965', '10967', '10969', '10997', '10999'):  
    #print(feature['properties']['STR_NAME'])
    #print(feature['properties']['DTV'])

    for position in feature['geometry']['coordinates']:

            df_traffic = df_traffic.append(
                {'str_name': feature['properties']['STR_NAME'],
                 'dtv':feature['properties']['DTV'] ,
                 'latitude':position[1],
                 'longitude':position[0]}, ignore_index=True)

In [78]:
df_traffic.groupby(['str_name','dtv']).mean().reset_index()


,str_name,dtv,latitude,longitude
0,-,9470,52.536666,13.640371
1,-,9720,52.530253,13.642771
2,-,20160,52.542840,13.633398
3,100 AD Charlottenburg Ausfahrt n Tegeler Weg,45798,52.532825,13.289235
4,100 AD Funkturm Verb-Fb v 100 Norden n A 115,28375,52.500659,13.274955
...,...,...,...,...
3845,Zwickauer Damm,7020,52.420945,13.481464
3846,ehemals 100 AS Schmargendorf Ausf n Dillenburg...,9496,52.469575,13.306668
3847,ehemals 100 AS Schmargendorf Ausfahrt n Schild...,15470,52.465825,13.312450
3848,ehemals 100 AS Schmargendorf Einf v Dillenburg...,8841,52.469525,13.307206
